# Real Book API

This is where I'll test how best to split up classes. As an overview, I'll need to implement:

- Song
	- Title
	- Key
	- Time Signature
	- Tempo
	- Measures
		- Notes
		- Chords

Ideally, the API will be able to display the song as a PDF or, if using the CLI, display the song in the terminal.

## Implementation

Let's deal with our imports first.

In [252]:
import math
import numbers
import re

from decimal import Decimal
from enum import Enum, EnumType
from fractions import Fraction
from pydantic import computed_field, BaseModel, BeforeValidator, Field, ValidationError
from typing import Annotated, Any, Dict, List, Literal, Tuple, Union

Let's create the lower level objects first:

- Notes
- Chords

## Notes

First, let's create the constant, acceptable items available for notes and chords.

In [253]:
whole = Fraction(32, 32)
whole_triplet = whole / 3

In [254]:
class NoteName(str, Enum):
	A_FLAT = "Ab"
	A_NATURAL = "A"
	A_SHARP = "A#"
	B_FLAT = "Bb"
	B_NATURAL = "B"
	C_FLAT = "Cb"
	C_NATURAL = "C"
	C_SHARP = "C#"
	D_FLAT = "Db"
	D_NATURAL = "D"
	D_SHARP = "D#"
	E_FLAT = "Eb"
	E_NATURAL = "E"
	F_FLAT = "Fb"
	F_NATURAL = "F"
	F_SHARP = "F#"
	G_FLAT = "Gb"
	G_NATURAL = "G"
	G_SHARP = "G#"

Can't forget about note lengths. Because we'll run into problems with fractions and Enums, we need to create a FractionEnumMeta.

In [255]:
class FractionEnumMeta(type(Fraction), EnumType):
	pass


class FractionEnum(Fraction, Enum, metaclass=FractionEnumMeta):
	def __new__(cls, numerator: Any = 0, denominator: Any = None):
		self = object.__new__(cls)

		if denominator is None:
			if type(numerator) is int:
				self._numerator = numerator
				self._denominator = 1
				return self
			elif isinstance(numerator, numbers.Rational):
				self._numerator = numerator.numerator
				self._denominator = numerator.denominator
				return self
			elif isinstance(numerator, (float, Decimal)):
				self._numerator, self._denominator = numerator.as_integer_ratio()
			else:
				raise TypeError("Argument should be a rational number.")
		elif type(numerator) is int is type(denominator):
			pass
		elif isinstance(numerator, numbers.Rational) and isinstance(
			denominator, numbers.Rational
		):
			numerator, denominator = (
				numerator.numerator * denominator.denominator,
				denominator.numerator * numerator.denominator,
			)
		else:
			raise TypeError("Both arguments should be rational instances.")

		if denominator == 0:
			raise ZeroDivisionError(f"Fraction({numerator}, 0)")

		g = math.gcd(numerator, denominator)
		if denominator < 0:
			g = -g
		numerator //= g
		denominator //= g
		self._numerator = numerator
		self._denominator = denominator
		return self

In [256]:
class NoteLength(FractionEnum):
	WHOLE = 32, 32
	HALF = 16, 32
	HALF_DOTTED = 24, 32
	QUARTER = 8, 32
	QUARTER_DOTTED = 12, 32
	SIXTEENTH = 4, 32
	SIXTEENTH_DOTTED = 6, 32
	THIRTYSECOND = 2, 32

In [257]:
class NoteOctave(int, Enum):
	THREE = 3
	FOUR = 4
	FIVE = 5
	SIX = 6

I think that we've got enough information to create a dedicated note class at this point.

In [258]:
class Note(BaseModel):
	name: NoteName
	length: NoteLength
	octave: NoteOctave

Here's how you'd use the `Note` class.

In [259]:
try:
	c_quarter_note = Note(name="C", length=Fraction(1, 4), octave=3)
except ValidationError as e:
	print(e)

## Chords

Let's look at chords now.

In [260]:
class ChordType(str, Enum):
	MAJOR = "Maj"
	MINOR = "Min"
	AUGMENTED = "Aug"
	DIMINISHED = "Dim"
	SUSPENDED_TWO = "Sus2"
	SUSPENDED_FOUR = "Sus4"
	SEVEN = "7"
	NINE = "9"
	ELEVEN = "11"
	THIRTEEN = "13"

In [261]:
class ChordQuality(str, Enum):
	FIVE_FLAT = "b5"
	FIVE_SHARP = "#5"
	SIX_FLAT = "b6"
	SIX = "6"
	SEVEN_FLAT = "b7"
	SEVEN = "7"
	NINE_FLAT = "b9"
	NINE = "9"
	NINE_SHARP = "#9"
	ELEVEN = "11"
	ELEVEN_SHARP = "#11"
	THIREEN_FLAT = "b13"
	THIRTEEN = "13"

Let's figure out a good way to combine `Quality` options.

In [262]:
QUALITIES_PATTERN = "|".join(
	re.escape(q.value)
	for q in sorted(ChordQuality, key=lambda x: len(x.value), reverse=True)
)


def parse_qualities(v: Any) -> List[ChordQuality]:
	if isinstance(v, str):
		matches = re.findall(QUALITIES_PATTERN, v)
		if matches:
			return matches
	return v


FlexibleQualities = Annotated[
	List[ChordQuality], BeforeValidator(parse_qualities), Field(max_length=2)
]

In [263]:
class Chord(BaseModel):
	root: NoteName
	chord_type: ChordType
	quality: FlexibleQualities = []
	length: NoteLength

	@computed_field
	@property
	def name(self) -> str:
		c_root = self.root.value
		c_type = self.chord_type.value
		c_qualities = "".join([q.value for q in self.quality])

		if self.quality and (c_type == self.quality[0].value in {"7", "9", "11", "13"}):
			return f"{c_root}{c_qualities}"
		return f"{c_root}{c_type}{c_qualities}"

Let's see how we'd use it.

In [264]:
try:
	c_seven_flat_nine_chord = Chord(
		root="C", chord_type="7", quality="#5b9", length=Fraction(1, 2)
	)
	print(c_seven_flat_nine_chord.name)
except ValidationError as e:
	print(e)

C7#5b9


Let's make sure the 7s cancel each other out properly.

In [265]:
try:
	c_nine = Chord(root="C", chord_type="7", quality="7", length=Fraction(1, 2))
	print(c_nine.name)
except ValidationError as e:
	print(e)

C7


## Songs

Figuring out how to handle measures, time signatures, etc. might be tricky.

In [266]:
class TimeSignature(FractionEnum):
	TWO_TWO = 2, 2
	THREE_TWO = 3, 2
	TWO_FOUR = 2, 4
	THREE_FOUR = 3, 4
	FOUR_FOUR = 4, 4
	FIVE_FOUR = 5, 4
	SIX_FOUR = 6, 4
	THREE_EIGHT = 3, 8
	FOUR_EIGHT = 4, 8
	FIVE_EIGHT = 5, 8
	SIX_EIGHT = 6, 8
	SEVEN_EIGHT = 7, 8
	NINE_EIGHT = 9, 8
	TWELVE_EIGHT = 12, 8

In [267]:
class Song(BaseModel):
	name: str
	key: NoteName
	signature: TimeSignature
	tempo: float = 120.0
	chord_measures: List[Chord]
	note_measures: List[Note]

	@computed_field
	@property
	def formatted_note_measures(self) -> List[Note]:
		formatted_notes = []
		measure = []
		tally = 1

		for note in self.note_measures:
			tally += note.length
			measure.append(note)
			if tally % self.signature.denominator == 0:
				formatted_notes.append(measure)
				measure = []
		
		return formatted_notes
	
	@computed_field
	@property
	def formatted_chord_measures(self) -> List[Chord]:
		formatted_chords = []
		measure = []
		tally = 1

		for chord in self.chord_measures:
			tally += chord.length
			measure.append(chord)
			if tally % self.signature.denominator == 0:
				formatted_chords.append(measure)
				measure = []
		
		return formatted_chords

Let's see how this might work.

In [268]:
chord_measures: List[Chord] = [
	Chord(root="C", length=Fraction(4, 4), chord_type="Maj", quality="7")
]

note_measures: List[Note] = [
	Note(name="C", length=Fraction(1, 4), octave=3),
	Note(name="E", length=Fraction(1, 4), octave=3),
	Note(name="G", length=Fraction(1, 4), octave=3),
	Note(name="B", length=Fraction(1, 4), octave=3),
]

c_major_seven_song = Song(
	name="C Major Seven Song",
	key="C",
	signature=Fraction(4, 4),
	tempo=120,
	chord_measures=chord_measures,
	note_measures=note_measures,
)

In [269]:
chord_measures: List[Chord] = [
	Chord(root="C", length=Fraction(4, 4), chord_type="Maj", quality="7"),  # 1
	Chord(root="F", length=Fraction(4, 4), chord_type="Maj", quality="7"),  # 2
	Chord(root="D", length=Fraction(1, 2), chord_type="Min", quality="9"),  # 3
	Chord(root="G", length=Fraction(1, 2), chord_type="7"),  # 3.5
	Chord(root="C", length=Fraction(4, 4), chord_type="Maj", quality="7"),  # 4
]

note_measures: List[Note] = [
	# Measure 1
	Note(name="C", length=Fraction(1, 4), octave=3),
	Note(name="E", length=Fraction(1, 4), octave=3),
	Note(name="G", length=Fraction(1, 4), octave=3),
	Note(name="B", length=Fraction(1, 4), octave=3),
	# Measure 2
	Note(name="F", length=Fraction(1, 4), octave=3),
	Note(name="A", length=Fraction(1, 4), octave=3),
	Note(name="C", length=Fraction(1, 4), octave=3),
	Note(name="E", length=Fraction(1, 4), octave=3),
	# Measure 3
	Note(name="D", length=Fraction(1, 4), octave=3),
	Note(name="F", length=Fraction(1, 4), octave=3),
	# Measure 3.5
	Note(name="G", length=Fraction(1, 4), octave=3),
	Note(name="B", length=Fraction(1, 4), octave=3),
	# Measure 4
	Note(name="C", length=Fraction(1, 4), octave=3),
	Note(name="E", length=Fraction(1, 4), octave=3),
	Note(name="G", length=Fraction(1, 4), octave=3),
	Note(name="B", length=Fraction(1, 4), octave=3),
]

test_song = Song(
	name="Test Song",
	key="C",
	signature=Fraction(4, 4),
	tempo=120,
	chord_measures=chord_measures,
	note_measures=note_measures,
)

for n in test_song.formatted_note_measures:
	notes = [i.name.value for i in n]
	print(notes)

['C', 'E', 'G', 'B']
['F', 'A', 'C', 'E']
['D', 'F', 'G', 'B']
['C', 'E', 'G', 'B']


Let's see if we can rewrite this without the Pylance errors.

In [270]:
test_song_data = {
	"name": "Test Song",
	"key": "C",
	"signature": Fraction(4, 4),
	"tempo": 120,
	"chord_measures": [
		{
			"root": "C",
			"chord_type": "Maj",
			"quality": "7",
			"length": Fraction(4, 4),
		},
		{
			"root": "F",
			"chord_type": "Maj",
			"quality": "7",
			"length": Fraction(4, 4),
		},
		{
			"root": "D",
			"chord_type": "Min",
			"quality": "9",
			"length": Fraction(2, 4),
		},
		{
			"root": "G",
			"chord_type": "7",
			"length": Fraction(2, 4),
		},
		{
			"root": "C",
			"chord_type": "Maj",
			"quality": "7",
			"length": Fraction(4, 4),
		},
	],
	"note_measures": [
		{
			"name": "C",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "E",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "G",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "B",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "F",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "A",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "C",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "E",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "D",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "F",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "G",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "B",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "C",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "E",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "G",
			"length": Fraction(1, 4),
			"octave": 3,
		},
		{
			"name": "B",
			"length": Fraction(1, 4),
			"octave": 3,
		},
	],
}

In [271]:
test_song = Song.model_validate(test_song_data)

How can we print this?

In [272]:
for idx, c in enumerate(test_song.formatted_chord_measures):
  c_names = [i.name for i in c]
  n_names = [i.name.value for i in test_song.formatted_note_measures[idx]]
  print(c_names)
  print(n_names)

['CMaj7']
['C', 'E', 'G', 'B']
['FMaj7']
['F', 'A', 'C', 'E']
['DMin9', 'G7']
['D', 'F', 'G', 'B']
['CMaj7']
['C', 'E', 'G', 'B']
